In [150]:
import pandas as pd
import numpy as np
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

Информация по талицам:

contractor_test - информация о контрагентах 

 - ITN - ИНН
 - name_contractor - название компании контрагента
 - form - передача форм собственности предприятий

contractor_test - информация о контранках

 - number - номер договора
 - date - дата заключения договора
 - ITN_contractor - ИНН контрагента, с которым заключен Договор
 - our_template	 - наш шаблон Договора (1 - Наш договор; 0 - Договор контрагента)
 - price - стоимость контракта 
 - object - объект строительства
 - gu - гарантийные удержания
 - gp - ген.услуги
 - prepayment - аванс (Доля от стоисоти договора)


In [240]:
# Информация о Контрактах

class WorkWithContract:
    data : pd.DataFrame
    copy: bool
    __instence = None
    
    
    def __new__(cls, *args, **kwargs):
        if cls.__instence == None:
            cls.__instence = super().__new__(cls)
        return  WorkWithContract.__instence
        
    
    def __init__(self, data, copy: bool = False):
        self.data = data
        self.info = {'number' : 'номер договора',
                'date': 'дата заключения договора в формате DD.MM.YYYY',
                'ITN_contractor' : 'ИНН контрагента, с которым заключен Договор',
                'price' : 'стоимость контракта', 
                'object' : 'объект строительства', 
                'our_template': 'наш шаблон Договора (1 - Наш договор; 0 - Договор контрагента)',
                'gu' : 'гарантийные удержания',
                'gp' : 'ген.услуги',
                'prepayment' : 'аванс (Доля от стоисоти договора)'}
        self.gu = 0.05
        self.gp = 0.02
        self.prepayment = 0.30
        if copy:
            self.data = data.copy()
        else:
            self.data = data
            
    
        
    def check_number(self, other):
        """Проверка на наличие номера Договора в Базе"""
        if self.data.isin({'number' : other}).any()[0]:
            print(f'Контракт c № {other} уже есть в Базе')
        else:
            print(f'Номер {other} свободен, можно использовать!')
            
            
    def our_template(self):
        pass
        
        
    def add_new(self):
        """Добавление нового контракта в Базу"""
        new_row = {}
        for i in self.data.columns:
            print(f'Введите информацию {self.info[i]}')
            a = input()
            if i == 'date':
                a = datetime.strptime(a, '%d.%m.%Y')
            elif i in ['price', 'gu', 'gp', 'prepayment']:
                a = float(a)
            new_row[i] = a
            if i == 'our_template':
                if a == '1':
                    new_row['gu'] = self.gu
                    new_row['gp'] = self.gp
                    new_row['prepayment'] = self.prepayment
                    print('Размер Гарантийных удержаний и Ген услуг проставлены согласно Стандартного шаблона Договора')
                    break            
        new_row_1 = pd.Series(data=new_row)
        self.data = self.data.append(new_row_1, ignore_index=True)
        print(f'Договор записан')
        return self.data
    

# Можно переносить его в отдельную Базу

    def drop_contract(self, other): 
        """Удаление Договора из базы по номеру Договора"""
        self.data.set_index('number', inplace = True)
        self.data.drop(other, inplace = True)
        print(f'Договор № {other} удален из базы!!!!')
        self.data.reset_index(inplace=True)
        return self.data
    
    def change_terms(self):
# Внесение изменений 
        pass

    
    def end_work(self):
        """Конец работы с документом и запись новой информации"""
#         today_day = datetime.today().strftime('%d-%m-%Y')
        name_file = 'contract_02-06-2022.csv'
        self.data.to_csv(name_file, index = False)
        print('Файл записан')
    
    
    def __del__(self):
        cls.__instence = None




In [226]:
# Информация о Контрагентах

class WorkWithContractor():
    data : pd.DataFrame
    __instence = None
    
    
    def __new__(cls, *args, **kwargs):
        if cls.__instence == None:
            cls.__instence = super().__new__(cls)
        return cls.__instence
    
    def __init__(self, data):
        self.data = data #База со сведеньями о Контрагентах
        self.info = {'ITN': 'ИНН контрагента',
                'name_contractor':'название компании контрагента',
                'form' : 'передача форм собственности предприятий'}
    
    def check_contractor(self):
        """Проверка на наличие контрагента в Базе по ИНН"""
        print(f'Введите ИНН контрагента')
        ITN_contractor = input()
        if self.data.isin({'ITN' : [ITN_contractor]}).any()[0]:
            print(f'Контрагент с ИНН {ITN_contractor} уже есть в Базе')
            return self.data[self.data['ITN'] == ITN_contractor]
        else:
            print(f'Контрагентa c ИНН {ITN_contractor} нет в Базе Контрагентов')
            
    def add_new(self):
        """Добавление нового контракта в Базу"""
        new_row = {}
        for i in self.data.columns:
            print(f'Введите информацию {self.info[i]}')
            a = input()
            new_row[i] = a     
        new_row_1 = pd.Series(data=new_row)
        self.data = self.data.append(new_row_1, ignore_index=True)
        print(f'Контрагент записан в Базу')
        return self.data
    
    def drop_contractor(self, other):     
        self.data.set_index('ITN', inplace = True)
        self.data.drop(other, inplace = True)
        print(f'Контрагент {other} удален из базы!!!!')
        self.data.reset_index(inplace=True)
        return self.data
            
    def end_work(self):
        """Конец работы с документом и запись новой информации"""
#         today_day = datetime.today().strftime('%d-%m-%Y')
        name_file = 'contractor_02-06-2022.csv'
        self.data.to_csv(name_file.format(today_day), index = False)
        print('Файл записан')
        
    def __del__(self):
        cls.__instence = None

In [272]:
data = pd.read_csv('contract_02-06-2022.csv')

In [269]:
contract = WorkWithContract(data)

In [270]:
contract.drop_contract('ЛТС-123')

Договор № ЛТС-123 удален из базы!!!!


,number,date,ITN_contractor,price,object,our_template,gu,gp,prepayment
0,1-TERRA-37,2020-08-21 00:00:00,7806564446,35000000.00,TERRA_1,1,0.05,0.02,0.3
1,1-TERRA-41,2020-09-08 00:00:00,7811060998,46577955.88,TERRA_1,1,0.05,0.02,0.3
2,2-TERRA-52,2020-09-26 00:00:00,7811060998,41838268.08,TERRA_2,1,0.05,0.02,0.3
3,B7NO-4169/4188,2020-12-01 00:00:00,7805223443,63331200.00,TERRA_1,0,0.00,0.0,1.0


In [261]:
contract.data

,number,date,ITN_contractor,price,object,our_template,gu,gp,prepayment
0,ЛТС-123,12.02.2021,7816659721,1300000.0,NEW_TIME,1,0.05,"0,02",0.0
1,1-TERRA-37,2020-08-21 00:00:00,7806564446,35000000.0,TERRA_1,1,0.05,0.02,0.3


In [266]:
contract.add_new()

Введите информацию номер договора
B7NO-4169/4188
Введите информацию дата заключения договора в формате DD.MM.YYYY
01.12.2020
Введите информацию ИНН контрагента, с которым заключен Договор
7805223443
Введите информацию стоимость контракта
63331200.00
Введите информацию объект строительства
TERRA_1
Введите информацию наш шаблон Договора (1 - Наш договор; 0 - Договор контрагента)
0
Введите информацию гарантийные удержания
0
Введите информацию ген.услуги
0
Введите информацию аванс (Доля от стоисоти договора)
1
Договор записан


,number,date,ITN_contractor,price,object,our_template,gu,gp,prepayment
0,ЛТС-123,12.02.2021,7816659721,1300000.00,NEW_TIME,1,0.05,"0,02",0.0
1,1-TERRA-37,2020-08-21 00:00:00,7806564446,35000000.00,TERRA_1,1,0.05,0.02,0.3
2,1-TERRA-41,2020-09-08 00:00:00,7811060998,46577955.88,TERRA_1,1,0.05,0.02,0.3
3,2-TERRA-52,2020-09-26 00:00:00,7811060998,41838268.08,TERRA_2,1,0.05,0.02,0.3
4,B7NO-4169/4188,2020-12-01 00:00:00,7805223443,63331200.00,TERRA_1,0,0.00,0.0,1.0


In [271]:
contract.end_work()

Файл записан


In [13]:
# Расчёт задолности по Объекту
class ObjectDebt(): 
    
    def __init__(self, data):
        self.data = data
    
# Расчёт задолности по Контрагенту


class CotractorDebt():

    def __init__(self, data):
        self.data = data

# Расчёт задолности по Контракту

class ContractDebt():
    pass

In [3]:
contractor = pd.read_csv('contractor_02-06-2022.csv.csv')